In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [2]:
# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device : {device}')

Using device : cuda


In [3]:
# cuDNN 최적화 
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True

In [4]:
# 모든 변수 CUDA 상태 일괄 확인

def check_all_cuda_status():
    '''현재 namespace의 모든 변수 CUDA 상태 확인'''

    # 전역 변수 가져오기
    global_vars = globals()

    cuda_vars = []
    cpu_vars = []
    non_tensor_vars = []

    for var_name, var_value in global_vars.items():
        # 시스템 변수 제외
        if var_name.startswith('_'):
            continue
            
        # PyTorch 텐서인지 확인
        if isinstance(var_value, torch.Tensor):
            if var_value.is_cuda:
                cuda_vars.append(var_name)
            else:
                cpu_vars.append(var_name)
        
        # PyTorch 모델인지 확인
        elif isinstance(var_value, nn.Module):
            try:
                if next(var_value.parameters()).is_cuda:
                    cuda_vars.append(f"{var_name} (model)")
                else:
                    cpu_vars.append(f"{var_name} (model)")
            except StopIteration:
                # 파라미터가 없는 모델
                non_tensor_vars.append(f"{var_name} (empty model)")
        
        # 기타 변수들
        else:
            non_tensor_vars.append(var_name)
    
    # 결과 출력
    print("=== CUDA 상태 확인 결과 ===")
    print(f"\n✅ CUDA에 있는 변수들 ({len(cuda_vars)}개):")
    for var in cuda_vars:
        print(f"  - {var}")
    
    print(f"\n❌ CPU에 있는 변수들 ({len(cpu_vars)}개):")
    for var in cpu_vars:
        print(f"  - {var}")
    
    print(f"\n➖ 텐서/모델이 아닌 변수들 ({len(non_tensor_vars)}개):")
    for var in non_tensor_vars[:10]:  # 처음 10개만 표시
        print(f"  - {var}")
    if len(non_tensor_vars) > 10:
        print(f"  ... 및 {len(non_tensor_vars) - 10}개 더")
    
    return cuda_vars, cpu_vars, non_tensor_vars

In [5]:
# check_all_cuda_status()

In [6]:
import gc
# cuda cache 정리
torch.cuda.empty_cache()
gc.collect()

30

In [7]:
# read in all the words
words = open('../data/names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [8]:
# build the vocabulary of characters and mapping to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [9]:
# shuffle up the words
import random
random.seed(42)
random.shuffle(words)

In [38]:
# build the dataset
block_size = 8 # context length : how many characters do we take to predict the next one?
def build_dataset(words):
    X,Y = [], []
    for w in words:
        # print(w)
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)  
            Y.append(ix)
            context = context[1:] + [ix] # crop and append
    X = torch.tensor(X) 
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X,Y

import random 
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])      # 80% training set
Xdev, Ydev = build_dataset(words[n1:n2])  # 10% dev=validation set
Xte, Yte = build_dataset(words[n2:])      # 10% test set

Xtr = Xtr.to(device)
Ytr = Ytr.to(device)
Xdev = Xdev.to(device)
Ydev = Ydev.to(device)
Xte = Xte.to(device)
Yte = Yte.to(device)

torch.Size([182437, 8]) torch.Size([182437])
torch.Size([22781, 8]) torch.Size([22781])
torch.Size([22928, 8]) torch.Size([22928])


In [39]:
for x,y in zip(Xtr[:20],Ytr[:20]):
    print(''.join(itos[ix.item()] for ix in x), '-->', itos[y.item()])

........ --> t
.......t --> a
......ta --> u
.....tau --> r
....taur --> e
...taure --> n
..tauren --> .
........ --> s
.......s --> u
......su --> l
.....sul --> e
....sule --> m
...sulem --> a
..sulema --> n
.suleman --> .
........ --> z
.......z --> e
......ze --> r
.....zer --> e
....zere --> n


In [ ]:
# Near copy paste of the layers we have developed in Part 3

# -----------------------------------------------------
class Linear:
    def __init__(self, fan_in, fan_out, bias = True):
        self.weight = torch.randn((fan_in, fan_out)) / fan_in**0.5 # note : kaiming init
        self.bias = torch.zeros(fan_out) if bias else None

    def __call__(self, x):
        self.out = x @ self.weight
        if self.bias is not None:
            self.out += self.bias
        return self.out
    
    def parameters(self):
        return [self.weight] + ([] if self.bias is None else [self.bias])
    
    def to(self, device):
        self.weight = self.weight.to(device)
        if self.bias is not None:
            self.bias = self.bias.to(device)
        return self
    
# --------------------------------------------------------------------
class BatchNorm1d:

    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True
        # parameters (trained with backprop)
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        # buffers (trained with a running 'momentum update')
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)

    def __call__(self, x):
        # calculate the forward pass
        if self.training:
            if x.ndim == 2:
                dim = 0
            elif x.ndim == 3:
                dim = (0,1)
            xmean = x.mean(0, keepdim=True) # batch mean
            xvar = x.var(0, keepdim=True, unbiased=True) # batch variance
        else:
            xmean = self.running_mean
            xvar = self.running_var
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
        self.out = self.gamma * xhat + self.beta
        # update the buffers
        if self.training:
            with torch.no_grad():
                self.running_mean = (1-self.momentum) * self.running_mean + self.momentum * xmean
                self.running_var = (1-self.momentum) * self.running_var + self.momentum * xvar
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]
    
    def to(self, device):
        self.gamma = self.gamma.to(device)
        self.beta = self.beta.to(device)
        self.running_mean = self.running_mean.to(device)
        self.running_var = self.running_var.to(device)
        return self
    
#------------------------------------------------------------------- 
class Tanh:
    def __call__(self,x):
        self.out = torch.tanh(x)
        return self.out
    
    def parameters(self):
        return []   
    
    def to(self, device):
        return self 

#-------------------------------------------------------------------- 
class Embedding:
    def __init__(self, num_embeddings, embedding_dim):
        self.weight = torch.randn((num_embeddings, embedding_dim))

    def __call__(self,IX):
        self.out = self.weight[IX]
        return self.out
    
    def parameters(self):
        return [self.weight]
    
    def to(self, device):
        self.weight = self.weight.to(device)
        return self
    
# --------------------------------------------------------------------
class FlattenConsecutive:

    def __init__(self, n):
        self.n = n

    def __call__(self,x):
        B, T, C = x.shape
        x = x.view(B, -1, C*n)
        self.out = x.view(x.shape[0], -1)
        return self.out
    # 33:30부터
    
    def parameters(self):
        return []  
    
    def to(self, device):
        return self 
    
# --------------------------------------------------------------------
class Sequential:

    def __init__(self, layers):
        '''받은 layer list를 객체의 속성으로 저장한다.'''
        self.layers = layers

    def __call__(self, x):
        '''객체를 함수처럼 호출하게 해준다.'''
        for layer in self.layers:
            # 첫 번째 레이어의 출력이 두 번째 레이어의 입력이 되는 식으로, x를 update하면서 연쇄적으로 처리한다.
            x = layer(x)
        self.out = x
        return self.out
    
    def parameters(self):
        # get parameters of all layers and stretch them out into one list
        return [p for layer in self.layers for p in layer.parameters()]
    
    def to(self, device):
        for layer in self.layers:
            layer.to(device)
        return self


In [41]:
torch.manual_seed(42);

In [42]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

model = Sequential([
    Embedding(vocab_size, n_embd),
    Flatten(),
    Linear(n_embd * block_size, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    Linear(n_hidden, vocab_size),
])

# 모든 layer를 device로 이동
model.to(device)

# parameter init
with torch.no_grad():
    model.layers[-1].weight *= 0.1 # last layer make less confident

parameters = model.parameters()
print(sum(p.nelement() for p in parameters)) # number of parameters in total

for p in parameters:
    p.requires_grad = True

for i, p in enumerate(parameters):
    p = p.to(device)  

22097


In [ ]:
ix = torch.randint(0, Xtr.shape[0], (4,)) # let's look at a batch of just 4 examples
Xb, Yb = Xtr[ix], Ytr[ix]
logits = model(Xb)
print(Xb.shape)
# [4,8] : 4 because we picked four numbers, 8 is block_size
Xb

torch.Size([4, 8])


tensor([[ 0,  0,  0,  0,  0,  0,  0, 14],
        [ 0,  0,  0, 19,  5, 14,  1, 25],
        [ 0, 15, 12, 25, 13, 16,  9,  1],
        [ 0,  0,  0, 20, 18,  9, 20, 15]], device='cuda:0')

In [ ]:
model.layers[0].out.shape # output of Embedding layer

torch.Size([4, 8, 10])

In [ ]:
model.layers[1].out.shape # output of Flatten layer

torch.Size([4, 80])

In [ ]:
model.layers[2].out.shape # output of Linear layer

torch.Size([4, 200])

In [ ]:
(torch.randn(4,4,20) @ torch.randn(20,200) + torch.randn(200)).shape
# How it works?
# In the Pytorch, the matrix multiplication only works on the last Dimension.
# (for upward example torch.randn(4,5,2,80), it is 80)
# and the dimension before it in the input tensor are left unchanged.

torch.Size([4, 200])

In [ ]:
e = torch.randn(4,8,10) # goal: want this to be (4, 4, 20) where consecutive 10-d vectors get concatenated


In [43]:
# same optimization as last time
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):

    # minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size,))
    ix = ix.to(device)
    Xb,Yb = Xtr[ix], Ytr[ix] # batch X,Y

    # forward pass
    logits = model(Xb)
    loss = F.cross_entropy(logits, Yb) # loss function

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # update: simple SGD
    lr = 0.1 if i < 150000 else 0.01 # step learning rate decay
    for p in parameters:
        p.data += -lr * p.grad

    # track stats
    if i % 10000 == 0: # print every once in a while
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())

    break

      0/ 200000: 3.3041


In [24]:
# torch.Size([200, 1000]) 열방향(행)으로 합산해서,총 200행, 평균을 구함.
# 매 1000번의 loss.log10()를 평균 낸것과 같음
# 150, 즉 150000번째에 learning decay
plt.plot(torch.tensor(lossi).view(-1,1000).mean(1))

RuntimeError: shape '[-1, 1000]' is invalid for input of size 1

In [35]:
# put layers into eval mode (needed for batchroom expecially)
for layer in model.layers:
    layer.training = False

In [36]:
# evaluate the loss
@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte),
  }[split]
  logits = model(x)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

train 3.292391777038574
val 3.291624069213867


### Performance log
- original (3 character context + 200 hidden neurons, 12k params): train 2.058, val 2.105
- context: 3 -> 8 (22k params): train 1.918, val 2.027

In [37]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # forward pass the neural net
      logits = model(torch.tensor([context]))
      probs = F.softmax(logits, dim=1)
      # sample from the distribution
      ix = torch.multinomial(probs, num_samples=1).item()
      # shift the context window and track the samples
      context = context[1:] + [ix]
      out.append(ix)
      # if we sample the special '.' token, break
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out)) # decode and print the generated word

vritajokyvgrzppeopkgbr.
eqwskmairiphqiojfibjxqivf.
btuupwhoqxgzmiirwxifbqoionyuqucicpmoph.
q.
smqolvhqsugnqmkfzwltv.
vlhcvqdstwdnzqorkeezuruedg.
hzdikhtptrynfpzykyrqjytrjyjjnfwkzgtaoyrzzrzrrojdgceenn.
qqzligtgjpfvddzarifpkzpkvgdcxslcncofrnclekelcfkgpplrgywuzjqckspcnpsjwzgkbcvxkdgacqotnlzxcrqzdkgvckzkl.
.
jcxa.
socssekuhdvqnf.
ejpmacbgm.
tiewwplkfxsgkmplduljhqyxztuepoyabrcvelvvkjzabzqmgkhnv.
vynpsetlgsldne.
hdhez.
paixoikvqjnasoxsufjtwnxcpwybftyzylaeicmrhrphmllgpugsxuguuflgcbbuusnzor.
hsneawwrwuisfwgrmnqgftlowzhgmgllhfmdmjdijzvhqclkeabbxikunojephk.
epftdrmibrzpxubijf.
fsxkrcgqxskimptqmjxmvoekhcb.
wgrccdftwclykglkthdtluexyvzjyudkfzyknvmkbxkakvlffvccehtwzlivfkdkddvn.
